In [1]:
#pip install nltk

In [2]:
import numpy as np
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
# Dropping all rows where text column is NaN
train.dropna(axis=0, how="any", thresh=None, subset=['text'], inplace=True)
#filling all nan rows with empty string
test = test.fillna(' ')

# Checking length of each article
length = []
for text in train['text']:
    length.append(len(str(text)))
train['length'] = length

# Removing outliers, it will reduce overfitting
train = train.drop(train['text'][train['length'] < 50].index, axis = 0)

In [5]:
# Secluding labels in a new pandas dataframe for supervised learning
train_labels = train['label']
# Splitting data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(train['text'], train_labels, test_size=0.2, random_state=0)

In [6]:
# Setting up Term Frequency - Inverse Document Frequency Vectorizer
tfidf = TfidfVectorizer(stop_words = 'english', max_df = 0.7,max_features=50)
# Fit and transform training set and transform test set
tfidf_train = tfidf.fit_transform(x_train) 
tfidf_test = tfidf.transform(x_test)
tfidf_test_final = tfidf.transform(test['text'])

In [7]:
# Setting up Passive Aggressive Classifier
pac = PassiveAggressiveClassifier()
# Fitting on the training set
pac.fit(tfidf_train, y_train)
print()

In [8]:
# Predicting on the test set
y_pred = pac.predict(tfidf_test)


In [9]:
def predict():
    tf_idf=input()
    if len(tf_idf.split(" ")) <= 50:
        print("Please enter a text of more than 50 words")
    tfidf_test = tfidf.transform([tf_idf]) 
    y_pred = pac.predict(tfidf_test)
    response = ""
    if y_pred[0] == 0:
        response = "Fake " 
    else:
        response = "Not Fake"
    print(response)

In [10]:
predict()

y_confidence = pac.decision_function(tfidf_test)
Please enter a text of more than 50 words
Not Fake


In [12]:
y_confidence = pac.decision_function(tfidf_test)
score = accuracy_score(y_test, y_pred)
f1score=f1_score(y_test, y_pred)
print(f'Accuracy: {round(score * 100, 2)}%')
print(f'F1 Score: {round(f1score * 100, 2)}%')
#print(classification_report(y_test,y_pred))

Accuracy: 81.63%
F1 Score: 82.1%
